HEM

In [1]:
import networkx as nx
import math
from collections import defaultdict

def construct_virtual_graph(hyperedges):
    virtual_graph = nx.DiGraph()
    
    for idx, hyperedge in enumerate(hyperedges):
        virtual_graph.add_node(idx)
    
    for i, edge1 in enumerate(hyperedges):
        for j, edge2 in enumerate(hyperedges):
            if i != j:
                if set(edge1['tail']) & set(edge2['head']):
                    virtual_graph.add_edge(i, j)
    
    for i, edge in enumerate(hyperedges):
        new_degree = calculate_hyperedge_degree(i, hyperedges, virtual_graph)
        virtual_graph.nodes[i]['weight'] = new_degree
    
    return virtual_graph

def calculate_hyperedge_degree(edge_idx, hyperedges, virtual_graph):
    current_edge = hyperedges[edge_idx]
    
    head_count = len(current_edge['head'])
    tail_count = len(current_edge['tail'])
    if tail_count == 0:
        base_ratio = head_count
    else:
        base_ratio = head_count / tail_count
    
    total_degree = 0.0
    has_intersection = False
    
    for j, other_edge in enumerate(hyperedges):
        if j == edge_idx:
            continue
            
        shared_nodes = set(current_edge['tail']) & set(other_edge['head'])
        
        if shared_nodes:
            has_intersection = True
            intersection_degree = len(shared_nodes) / len(current_edge['tail']) if len(current_edge['tail']) > 0 else 0
            
            other_head_count = len(other_edge['head'])
            other_tail_count = len(other_edge['tail'])
            if other_tail_count == 0:
                other_ratio = other_head_count
            else:
                other_ratio = other_head_count / other_tail_count
        
            if other_ratio == 0 or intersection_degree == 0:
                total_degree += base_ratio
            else:
                total_degree += base_ratio * (1 + other_ratio * intersection_degree)
    
    if not has_intersection:
        total_degree = base_ratio
    
    return total_degree

class HypergraphHEM:
    def __init__(self, virtual_graph, hyperedges, alpha=1.0, lambda_param=0.5, max_order=2):
        self.virtual_graph = virtual_graph
        self.hyperedges = hyperedges
        self.alpha = alpha
        self.lambda_param = lambda_param
        self.max_order = max_order
        self.neighbor_cache = {}

    def get_k_order_neighbors(self, node, order, direction='in'):
        key = (node, order, direction)
        if key not in self.neighbor_cache:
            self.neighbor_cache[key] = self._find_k_order_neighbors(node, order, direction)
        return self.neighbor_cache[key]

    def _find_k_order_neighbors(self, node, order, direction):
        if order == 0:
            return set([node])

        visited = set([node])
        current_level = set([node])

        for _ in range(order):
            next_level = set()
            for current_node in current_level:
                if direction == 'in':
                    next_level.update(self.virtual_graph.predecessors(current_node))
                else:
                    next_level.update(self.virtual_graph.successors(current_node))

            next_level -= visited

            if not next_level:
                break

            visited.update(next_level)
            current_level = next_level

        return next_level

    def calculate_k_order_entropy(self, node, order, direction):
        neighbors = self.get_k_order_neighbors(node, order, direction)

        if not neighbors:
            return 0.0

        total_degree = 0.0
        for neighbor in neighbors:
            if direction == 'in':
                degrees = [self.virtual_graph.nodes[n].get('weight', self.virtual_graph.out_degree(n)) for n in self.virtual_graph.predecessors(neighbor)]
            else:
                degrees = [self.virtual_graph.nodes[n].get('weight', self.virtual_graph.in_degree(n)) for n in self.virtual_graph.successors(neighbor)]

            total_degree += sum(degrees)

        if total_degree == 0:
            return 0.0

        entropy = 0.0

        for neighbor in neighbors:
            if direction == 'in':
                neighbor_degrees = [self.virtual_graph.nodes[n].get('weight', self.virtual_graph.out_degree(n)) for n in self.virtual_graph.predecessors(neighbor)]
            else:
                neighbor_degrees = [self.virtual_graph.nodes[n].get('weight', self.virtual_graph.in_degree(n)) for n in self.virtual_graph.successors(neighbor)]

            neighbor_degree_sum = sum(neighbor_degrees)
            p = neighbor_degree_sum / total_degree

            if p > 0:
                entropy -= p * math.log(p)

        return entropy

    def calculate_total_entropy(self, node):
        E_in = 0.0
        E_out = 0.0

        for k in range(1, self.max_order + 1):
            decay_factor = math.exp(-self.lambda_param * (k - 1))
            
            E_in_k = self.calculate_k_order_entropy(node, k, 'in')
            E_out_k = self.calculate_k_order_entropy(node, k, 'out')
            
            E_in += decay_factor * E_in_k
            E_out += decay_factor * E_out_k

        NIA = self.alpha * E_in + (1 - self.alpha) * E_out
        return E_in, E_out, NIA

    def calculate_NEM(self, node):
        in_neighbors = self.get_k_order_neighbors(node, 1, 'in')
        NEM = 0.0

        for neighbor in in_neighbors:
            E_in, E_out, NIA = self.calculate_total_entropy(neighbor)
            NEM += NIA+1

        return NEM

    def rank_nodes(self):
        nem_scores = {}
        for edge_idx, edge in enumerate(self.hyperedges):
            nem_scores[edge_idx] = self.calculate_NEM(edge_idx)

        node_impact = defaultdict(float)
        head_node_sharing = defaultdict(int)
        tail_node_sharing = defaultdict(int)
        bridge_nodes = defaultdict(int)

        for edge_idx, nem_score in nem_scores.items():
            hyperedge = self.hyperedges[edge_idx]
            
            num_head = len(hyperedge['head'])
            num_tail = len(hyperedge['tail'])
            
            head_weight = nem_score / num_head if num_head > 0 else 0
            tail_weight = nem_score / num_tail if num_tail > 0 else 0

            for head_node in hyperedge['head']:
                head_node_sharing[head_node] += 1
            for tail_node in hyperedge['tail']:
                tail_node_sharing[tail_node] += 1

            for head_node in hyperedge['head']:
                node_impact[head_node] += head_weight
            for tail_node in hyperedge['tail']:
                node_impact[tail_node] += tail_weight

            for head_node in hyperedge['head']:
                for other_edge_idx, other_hyperedge in enumerate(self.hyperedges):
                    if other_edge_idx != edge_idx and head_node in other_hyperedge['tail']:
                        bridge_nodes[head_node] += 1
            for tail_node in hyperedge['tail']:
                for other_edge_idx, other_hyperedge in enumerate(self.hyperedges):
                    if other_edge_idx != edge_idx and tail_node in other_hyperedge['head']:
                        bridge_nodes[tail_node] += 1

        for node in node_impact:
            shared_head_count = head_node_sharing[node]
            shared_tail_count = tail_node_sharing[node]

            if shared_head_count > 1:
                node_impact[node] += 0.3 * math.sqrt(shared_head_count)
            if shared_tail_count > 1:
                node_impact[node] += 0.2 * math.sqrt(shared_tail_count)
            
            if node in bridge_nodes:
                connected_edges = []
                for edge_idx, hyperedge in enumerate(self.hyperedges):
                    if node in hyperedge['head'] or node in hyperedge['tail']:
                        connected_edges.append(edge_idx)
                
                total_size_sum = 0.0
                edge_count = len(connected_edges)
                
                for i in range(len(connected_edges)):
                    for j in range(i + 1, len(connected_edges)):
                        edge1 = self.hyperedges[connected_edges[i]]
                        edge2 = self.hyperedges[connected_edges[j]]
                        
                        size1 = len(set(edge1['head']) | set(edge1['tail']))
                        size2 = len(set(edge2['head']) | set(edge2['tail']))
                        
                        size_sum = size1 + size2
                        weighted_score = math.exp(edge_count / size_sum)
                        total_size_sum += weighted_score
                
                node_impact[node] += total_size_sum

        ranked_node_ids = sorted(node_impact.items(), key=lambda x: x[1], reverse=True)
        
        return ranked_node_ids

if __name__ == "__main__":
    hyperedges = [
        {'head': [1, 2], 'tail': [3, 4]},
        {'head': [6, 7, 8], 'tail': [5, 22]},
        {'head': [3, 4, 5], 'tail': [9, 10]},
        {'head': [9, 10], 'tail': [11]},
        {'head': [12, 13], 'tail': [1, 2]},
        {'head': [9, 10, 19, 20], 'tail': [14, 15]},
        {'head': [14], 'tail': [16, 17, 18]},
        {'head': [21], 'tail': [19, 20]},
        {'head': [22, 23], 'tail': [21]},
        {'head': [15], 'tail': [24, 25]},
        {'head': [16, 17], 'tail': [26,27,28]},
        {'head': [18], 'tail': [29, 30]},
        {'head': [24], 'tail': [31, 32]},
        {'head': [24, 25], 'tail': [33, 34]},
        {'head': [27, 28, 29], 'tail': [35, 36]},
        {'head': [33, 34], 'tail': [37, 38, 39]},
        {'head': [40, 41], 'tail': [14]}
    ]

    virtual_graph = construct_virtual_graph(hyperedges)
    hypergraph_hem = HypergraphHEM(virtual_graph, hyperedges, alpha=0.5, lambda_param=0.5)
    
    print("\n===  HIC  ===")
    for i, edge in enumerate(hyperedges):
        NEM_value = hypergraph_hem.calculate_NEM(i)
        print(f"{i}: HIC  = {NEM_value:.6f}")


===  HIC  ===
0: HIC  = 1.000000
1: HIC  = 0.000000
2: HIC  = 2.430200
3: HIC  = 1.210194
4: HIC  = 0.000000
5: HIC  = 2.420388
6: HIC  = 3.338510
7: HIC  = 1.000000
8: HIC  = 1.430200
9: HIC  = 2.128303
10: HIC  = 1.523354
11: HIC  = 1.523354
12: HIC  = 1.176780
13: HIC  = 1.176780
14: HIC  = 2.000000
15: HIC  = 1.000000
16: HIC  = 0.000000
